In [1]:
jupyter nbconvert \
  --ClearMetadataPreprocessor.enabled=True \
  --inplace \
  "FINAL PROJECT (NL -_ SQL -_ Visualizations) (1).ipynb"

jupyter nbconvert \
  --ClearMetadataPreprocessor.enabled=True \
  --inplace \
  "FINAL GENERATIVE AI PROJECT (1).ipynb"


SyntaxError: invalid syntax (959349404.py, line 1)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# 1. Load the dataset
file_path = "/content/drive/MyDrive/Credit Card Data.csv"   # update path as needed
df = pd.read_csv(file_path)

# 2. Basic cleaning
# Ensure correct dtypes
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

# Create some helper features
df['age'] = (df['trans_date_trans_time'].dt.year - df['dob'].dt.year)
df['tx_hour'] = df['trans_date_trans_time'].dt.hour
df['tx_dayofweek'] = df['trans_date_trans_time'].dt.dayofweek  # 0=Mon

# Sort by user + time
df = df.sort_values(['cc_num', 'trans_date_trans_time']).reset_index(drop=True)


In [3]:
# Filter to legitimate transactions
df_legit = df[df['is_fraud'] == 0].copy()

def build_user_profiles(df_legit):
    # Basic numeric stats
    agg_numeric = df_legit.groupby('cc_num').agg(
        n_legit_tx=('amt', 'count'),
        total_spent=('amt', 'sum'),
        mean_spent=('amt', 'mean'),
        std_spent=('amt', 'std'),
        min_spent=('amt', 'min'),
        max_spent=('amt', 'max'),
        median_spent=('amt', 'median'),
        age_mean=('age', 'mean'),
        city_pop_mean=('city_pop', 'mean')
    )

    # Category distribution (% of transactions by category)
    cat_counts = (
        df_legit.groupby(['cc_num', 'category'])['amt']
        .count()
        .unstack(fill_value=0)
    )
    cat_pct = cat_counts.div(cat_counts.sum(axis=1), axis=0)
    cat_pct.columns = [f"cat_pct_{c}" for c in cat_pct.columns]

    # Time-of-day distribution
    # Buckets: night(0-6), morning(6-12), afternoon(12-18), evening(18-24)
    def time_bucket(h):
        if 0 <= h < 6:
            return 'night'
        elif 6 <= h < 12:
            return 'morning'
        elif 12 <= h < 18:
            return 'afternoon'
        else:
            return 'evening'

    df_legit['time_bucket'] = df_legit['tx_hour'].apply(time_bucket)

    tb_counts = (
        df_legit.groupby(['cc_num', 'time_bucket'])['amt']
        .count()
        .unstack(fill_value=0)
    )
    tb_pct = tb_counts.div(tb_counts.sum(axis=1), axis=0)
    tb_pct.columns = [f"tb_pct_{c}" for c in tb_pct.columns]

    # Geography: main state/city, avg lat/long
    geo_agg = df_legit.groupby('cc_num').agg(
        main_state=('state', lambda x: x.value_counts().idxmax()),
        main_city=('city', lambda x: x.value_counts().idxmax()),
        mean_lat=('lat', 'mean'),
        mean_long=('long', 'mean')
    )

    # Merge all
    user_profiles = (
        agg_numeric
        .join(cat_pct, how='left')
        .join(tb_pct, how='left')
        .join(geo_agg, how='left')
    ).reset_index()  # keep cc_num as a column

    return user_profiles

user_profiles = build_user_profiles(df_legit)
user_profiles.head()


,cc_num,n_legit_tx,total_spent,mean_spent,std_spent,min_spent,max_spent,median_spent,age_mean,city_pop_mean,...,cat_pct_shopping_pos,cat_pct_travel,tb_pct_afternoon,tb_pct_evening,tb_pct_morning,tb_pct_night,main_state,main_city,mean_lat,mean_long
0,60416207185,1509,83143.74,55.098569,120.422174,1.02,3075.09,36.66,33.277005,1645.0,...,0.098741,0.019881,0.361829,0.332671,0.157058,0.148443,WY,Fort Washakie,43.0048,-108.8964
1,60422928733,1519,98140.02,64.608308,83.453978,1.04,1290.37,52.15,77.276498,46944.0,...,0.099408,0.036208,0.205398,0.196182,0.292298,0.306122,SC,North Augusta,33.6028,-81.9748
2,60423098130,500,54075.55,108.151100,1211.999695,1.01,27119.77,34.68,61.266000,83.0,...,0.052000,0.044000,0.258000,0.292000,0.236000,0.214000,OK,Amorita,36.9412,-98.2458
3,60427851591,514,49483.82,96.272023,90.327352,20.74,569.40,76.36,43.278210,2142.0,...,0.091440,0.040856,0.268482,0.233463,0.284047,0.214008,OK,Burns Flat,35.3492,-99.1880
4,60487002085,496,25160.11,50.726028,65.843969,1.02,750.39,35.64,47.360887,233060.0,...,0.070565,0.028226,0.332661,0.344758,0.167339,0.155242,MS,Jackson,32.3739,-90.1293


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

# Merge user profiles back to all transactions
df_model = df.merge(
    user_profiles,
    on='cc_num',
    how='left',
    suffixes=('', '_prof')
)

# Select columns for the classifier
feature_cols_cat = ['category', 'state', 'city', 'job', 'gender']
feature_cols_num = [
    'amt', 'age', 'city_pop', 'tx_hour', 'tx_dayofweek',
    'n_legit_tx', 'total_spent', 'mean_spent', 'std_spent',
    'min_spent', 'max_spent', 'median_spent',
    'mean_lat', 'mean_long'
] + [c for c in df_model.columns if c.startswith('cat_pct_') or c.startswith('tb_pct_')]

# Label encode categorical vars
df_model_enc = df_model.copy()
label_encoders = {}

for col in feature_cols_cat:
    le = LabelEncoder()
    # Get all unique values from original data
    original_classes = df_model_enc[col].astype(str).unique()
    # Add a placeholder for unknown values that might appear in synthetic data
    # Ensuring 'UNKNOWN_CATEGORY' is part of the classes the encoder knows
    combined_classes = np.append(original_classes, 'UNKNOWN_CATEGORY')
    le.fit(combined_classes) # Fit on all possible original + 'UNKNOWN_CATEGORY'

    df_model_enc[col] = le.transform(df_model_enc[col].astype(str))
    label_encoders[col] = le

X = df_model_enc[feature_cols_num + feature_cols_cat]
y = df_model_enc['is_fraud'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer # Import SimpleImputer

# Impute missing values with the median (fit on training data)
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)

# Define three baseline models
models_baseline = {
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        n_jobs=-1,
        random_state=42,
        class_weight='balanced'  # handle class imbalance
    ),
    "GradientBoosting": GradientBoostingClassifier(
        random_state=42
    ),
    "LogisticRegression": LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ),
}

baseline_results = {}
fpr_baseline = {}

for name, clf in models_baseline.items():
    print(f"\n=== {name} baseline (original data only) ===")
    # Use imputed data for training and prediction
    clf.fit(X_train_imputed, y_train)
    y_pred = clf.predict(X_test_imputed)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)

    fpr_baseline[name] = fpr
    baseline_results[name] = {
        "confusion_matrix": cm,
        "fpr": fpr,
    }

    print("Confusion matrix:\n", cm)
    print(f"Baseline FPR ({name}):", fpr)
    print(classification_report(y_test, y_pred, digits=4))

# For backward compatibility
clf_baseline = models_baseline["RandomForest"]
fpr_baseline_rf = fpr_baseline["RandomForest"]


=== RandomForest baseline (original data only) ===
Confusion matrix:
 [[257805     29]
 [   407   1094]]
Baseline FPR (RandomForest): 0.00011247546871242737
              precision    recall  f1-score   support

           0     0.9984    0.9999    0.9992    257834
           1     0.9742    0.7288    0.8338      1501

    accuracy                         0.9983    259335
   macro avg     0.9863    0.8644    0.9165    259335
weighted avg     0.9983    0.9983    0.9982    259335


=== GradientBoosting baseline (original data only) ===
Confusion matrix:
 [[257716    118]
 [   434   1067]]
Baseline FPR (GradientBoosting): 0.0004576588037264286
              precision    recall  f1-score   support

           0     0.9983    0.9995    0.9989    257834
           1     0.9004    0.7109    0.7945      1501

    accuracy                         0.9979    259335
   macro avg     0.9494    0.8552    0.8967    259335
weighted avg     0.9978    0.9979    0.9977    259335


=== LogisticRegression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
def profile_to_text(row):
    # Use safe fills for NaNs
    main_state = row.get('main_state', 'Unknown')
    main_city = row.get('main_city', 'Unknown')
    mean_spent = row.get('mean_spent', 0)
    max_spent = row.get('max_spent', 0)
    n_tx = row.get('n_legit_tx', 0)

    # a few top category prefs
    cat_cols = [c for c in user_profiles.columns if c.startswith('cat_pct_')]
    top_cats = sorted(
        [(c.replace('cat_pct_', ''), row[c]) for c in cat_cols],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    cat_str = ", ".join([f"{name} ({pct:.0%})" for name, pct in top_cats if pct > 0])

    profile = (
        f"User {int(row['cc_num'])} lives mainly in {main_city}, {main_state}. "
        f"They have {int(n_tx)} legitimate transactions. "
        f"The average transaction amount is ${mean_spent:.2f} and max is ${max_spent:.2f}. "
    )
    if cat_str:
        profile += f"The main spending categories are: {cat_str}. "
    return profile

user_profiles['profile_text'] = user_profiles.apply(profile_to_text, axis=1)


In [9]:
df_fraud = df[df['is_fraud'] == 1].copy()

# Join profiles to fraud rows
df_fraud = df_fraud.merge(
    user_profiles[['cc_num', 'profile_text']],
    on='cc_num',
    how='left'
)

def tx_to_text(row):
    # Convert a single transaction row into a structured text / pseudo-JSON
    return (
        "{"
        f" 'trans_date_trans_time': '{row['trans_date_trans_time']}',"
        f" 'cc_num': '{int(row['cc_num'])}',"
        f" 'merchant': '{row['merchant']}',"
        f" 'category': '{row['category']}',"
        f" 'amt': {row['amt']:.2f},"
        f" 'city': '{row['city']}',"
        f" 'state': '{row['state']}',"
        f" 'zip': '{row['zip']}',"
        f" 'lat': {row['lat']},"
        f" 'long': {row['long']},"
        f" 'city_pop': {row['city_pop']},"
        f" 'job': '{row['job']}',"
        f" 'dob': '{row['dob']}',"
        f" 'unix_time': {row['unix_time']},"
        f" 'merch_lat': {row['merch_lat']},"
        f" 'merch_long': {row['merch_long']},"
        f" 'merch_zipcode': '{row['merch_zipcode']}',"
        f" 'is_fraud': 1"
        " }"
    )

prompts = []
completions = []

for _, row in df_fraud.iterrows():
    profile_text = row['profile_text']
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    completion = tx_to_text(row)
    prompts.append(prompt)
    completions.append(completion)

train_df_llm = pd.DataFrame({
    "prompt": prompts,
    "completion": completions
})
train_df_llm.head()


,prompt,completion
0,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 01:32:5...
1,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 02:42:2...
2,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-01 23:06:5...
3,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-02 22:10:3...
4,You are a fraud data generator. Given the foll...,{ 'trans_date_trans_time': '2019-03-02 22:10:5...


In [10]:
!pip install -q transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 25.9 MB/s eta 0:00:00


In [11]:
from datasets import Dataset

def format_example(ex):
    # Simple instruction-tuning style
    text = ex["prompt"] + ex["completion"]
    return {"text": text}

hf_ds = Dataset.from_pandas(train_df_llm[['prompt', 'completion']])
hf_ds = hf_ds.map(format_example, remove_columns=['prompt', 'completion'])


Map:   0%|          | 0/7506 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

model_name = "microsoft/phi-2"  # or another small causal LM

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_ds = hf_ds.map(tokenize_fn, batched=True, remove_columns=['text'])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Use BitsAndBytesConfig as recommended by the warning and for better device handling
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16, # or torch.bfloat16 if GPU supports it
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0} # Explicitly set device to GPU 0
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # adjust depending on model
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./fraud-llm",
    per_device_train_batch_size=2, # Reduced batch size
    gradient_accumulation_steps=4,
    num_train_epochs=0.5,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_total_limit=2,
    save_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator
)

trainer.train()

# Save adapter
trainer.save_model("./fraud-llm")
tokenizer.save_pretrained("./fraud-llm")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omkargp1 (omkargp1-umbc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.649000
100,0.904300
150,0.861000
200,0.843100
250,0.833200
300,0.824400
350,0.820500
400,0.815100
450,0.812000


('./fraud-llm/tokenizer_config.json',
 './fraud-llm/special_tokens_map.json',
 './fraud-llm/vocab.json',
 './fraud-llm/merges.txt',
 './fraud-llm/added_tokens.json',
 './fraud-llm/tokenizer.json')

In [13]:
import torch
import ast  # to safely parse the Python dict string

# Reload model + tokenizer (with LoRA)
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./fraud-llm")
tokenizer = AutoTokenizer.from_pretrained("./fraud-llm")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_fraud_for_user(profile_text, num_samples=10, max_new_tokens=256):
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        num_return_sequences=num_samples
    )

    gen_texts = []
    for i in range(num_samples):
        text = tokenizer.decode(outputs[i], skip_special_tokens=True)
        # Extract only the dictionary part (after the prompt)
        dict_str = text.split("FRAUD TRANSACTION:\n")[-1].strip()
        gen_texts.append(dict_str)
    return gen_texts


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
import torch
import ast  # to safely parse the Python dict string

# Reload model + tokenizer (with LoRA)
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./fraud-llm")
tokenizer = AutoTokenizer.from_pretrained("./fraud-llm")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_fraud_for_user(profile_text, num_samples=3, max_new_tokens=256):
    prompt = (
        "You are a fraud data generator. Given the following user profile, "
        "generate ONE realistic fraudulent credit card transaction for this user. "
        "Return ONLY a Python-style dictionary with keys matching the dataset, "
        "and set 'is_fraud' to 1.\n\n"
        f"USER PROFILE:\n{profile_text}\n\n"
        "FRAUD TRANSACTION:\n"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        num_return_sequences=num_samples
    )

    gen_texts = []
    for i in range(num_samples):
        text = tokenizer.decode(outputs[i], skip_special_tokens=True)
        # Extract only the dictionary part (after the prompt)
        dict_str = text.split("FRAUD TRANSACTION:\n")[-1].strip()
        gen_texts.append(dict_str)
    return gen_texts


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
synthetic_rows = []

# Example: generate for 10% of users to start
subset_profiles = user_profiles.sample(frac=0.1, random_state=42)

for _, row in subset_profiles.iterrows():
    profile_text = row['profile_text']
    cc_num = row['cc_num']
    gen_list = generate_fraud_for_user(profile_text, num_samples=2)

    for dict_str in gen_list:
        try:
            tx_dict = ast.literal_eval(dict_str)
            tx_dict['cc_num'] = cc_num  # ensure correct user id
            # enforce fraud label
            tx_dict['is_fraud'] = 1
            synthetic_rows.append(tx_dict)
        except Exception as e:
            print("Parse error, skipping:", e)

df_synth = pd.DataFrame(synthetic_rows)
df_synth.head()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: invalid syntax (<unknown>, line 3)
Parse error, skipping: invalid syntax (<unknown>, line 3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: unterminated string literal (detected at line 6) (<unknown>, line 6)
Parse error, skipping: unterminated triple-quoted string literal (detected at line 10) (<unknown>, line 5)
Parse error, skipping: invalid syntax (<unknown>, line 3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: unterminated string literal (detected at line 7) (<unknown>, line 7)
Parse error, skipping: invalid syntax (<unknown>, line 3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parse error, skipping: invalid syntax (<unknown>, line 3)
Parse error, skipping: invalid syntax (<unknown>, line 3)


,trans_date_trans_time,cc_num,merchant,category,amt,city,state,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,merch_zipcode,is_fraud
0,2019-03-04 23:41:14,3512365128314616,fraud_Schneider-Reed,grocery_pos,371.93,Arnold,MO,63111,38.2382,-92.6767,3814,Futures trading portfolio manager,1989-06-01 00:00:00,1327334554,37.483518,-92.077099,63935.0,1
1,2019-11-17 23:58:59,3563837241599446,fraud_Lueck and Son,grocery_pos,296.62,Newberg,OR,97103,44.0731,-122.2423,120862,Social policy adviser,1981-02-12 00:00:00,1347292669,43.752227,-122.140914,97106.0,1
2,2019-10-01 22:40:30,3568255211412877,"fraud_Kovacevich, Sime and Koss",grocery_pos,325.95,Leo,IN,47401,39.5708,-87.1499,1426,"Musician, classical",1939-09-04 00:00:00,1338669510,39.132234,-86.965399,48456.0,1
3,2020-02-27 22:48:17,4989847570577635369,fraud_Gruening-Hollister,home,6.24,Prosperity,SC,29619,32.8014,-81.6017,1523,"Journalist, film",1982-04-03 00:00:00,1368492097,32.180712,-82.817666,29455.0,1
4,2020-01-06 03:04:47,3501942333500073,"fraud_Dunn, Tost and Reising",misc_net,741.69,Phoenix,AZ,85001,33.7332,-112.6457,6371300,Healthcare professional,1958-03-25 00:00:00,1353293577,34.539286,-112.283897,86116.0,1


In [16]:
# Ensure all expected columns exist
expected_cols = df.columns  # from original dataset

for col in expected_cols:
    if col not in df_synth.columns:
        df_synth[col] = np.nan

df_synth = df_synth[expected_cols]


In [17]:
# Combine original training set rows (using indices from earlier split) with synthetic rows
# First, we reconstruct X_train / y_train with indices

train_indices = X_train.index
df_train_original = df_model_enc.loc[train_indices].copy()

# Re-encode synthetic data with the same label encoders
df_synth_enc = df_synth.copy()

# Convert dates and create helper features if needed, matching preprocessing
df_synth_enc['trans_date_trans_time'] = pd.to_datetime(df_synth_enc['trans_date_trans_time'])
df_synth_enc['dob'] = pd.to_datetime(df_synth_enc['dob'])
df_synth_enc['age'] = df_synth_enc['trans_date_trans_time'].dt.year - df_synth_enc['dob'].dt.year
df_synth_enc['tx_hour'] = df_synth_enc['trans_date_trans_time'].dt.hour
df_synth_enc['tx_dayofweek'] = df_synth_enc['trans_date_trans_time'].dt.dayofweek

# Merge profile features for synthetic rows too
df_synth_enc = df_synth_enc.merge(
    user_profiles,
    on='cc_num',
    how='left',
    suffixes=('', '_prof')
)

# Encode categorical features with existing label encoders
for col, le in label_encoders.items():
    # Ensure the column is of string type before processing to prevent FutureWarning
    df_synth_enc[col] = df_synth_enc[col].astype(str)

    # Replace unseen labels in df_synth_enc with 'UNKNOWN_CATEGORY' before transforming
    # Use .isin() for boolean indexing for efficiency and to handle potential NaNs safely
    unseen_mask = ~df_synth_enc[col].isin(le.classes_)
    if unseen_mask.any(): # Check if there are any unseen labels
        df_synth_enc.loc[unseen_mask, col] = 'UNKNOWN_CATEGORY'
    df_synth_enc[col] = le.transform(df_synth_enc[col])

# Build X_synth, y_synth
X_synth = df_synth_enc[feature_cols_num + feature_cols_cat]
y_synth = df_synth_enc['is_fraud'].astype(int)

# Concatenate training data
X_train_aug = pd.concat([X_train, X_synth], axis=0)
y_train_aug = pd.concat([y_train, y_synth], axis=0)

In [19]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Reuse the imputer from the baseline step (fitted on X_train) to transform
# the augmented training data. X_test_imputed is already available.
X_train_aug_imputed = pd.DataFrame(imputer.transform(X_train_aug), columns=X_train_aug.columns, index=X_train_aug.index)

# Define the same three model types for training on augmented data
models_augmented = {
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        n_jobs=-1,
        random_state=42,
        class_weight='balanced'
    ),
    "GradientBoosting": GradientBoostingClassifier(
        random_state=42
    ),
    "LogisticRegression": LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ),
}

augmented_results = {}
fpr_augmented = {}

for name, clf in models_augmented.items():
    print(f"\n=== {name} with synthetic data (augmented training set) ===")
    # Use the imputed augmented training data
    clf.fit(X_train_aug_imputed, y_train_aug)
    # Use the imputed test data from the baseline step
    y_pred_aug = clf.predict(X_test_imputed)

    cm_aug = confusion_matrix(y_test, y_pred_aug)
    tn_a, fp_a, fn_a, tp_a = cm_aug.ravel()
    fpr_a = fp_a / (fp_a + tn_a)

    fpr_augmented[name] = fpr_a
    augmented_results[name] = {
        "confusion_matrix": cm_aug,
        "fpr": fpr_a,
    }

    print("Confusion matrix (augmented):\n", cm_aug)
    print(f"Augmented FPR ({name}):", fpr_a)

    # Compare against the corresponding baseline FPR if available
    base_fpr = fpr_baseline.get(name) if isinstance(fpr_baseline, dict) else None
    if base_fpr is not None:
        print("Change in FPR vs baseline:", fpr_a - base_fpr)

    print(classification_report(y_test, y_pred_aug, digits=4))


=== RandomForest with synthetic data (augmented training set) ===
Confusion matrix (augmented):
 [[257807     27]
 [   399   1102]]
Augmented FPR (RandomForest): 0.00010471853983570825
Change in FPR vs baseline: -7.756928876719125e-06
              precision    recall  f1-score   support

           0     0.9985    0.9999    0.9992    257834
           1     0.9761    0.7342    0.8380      1501

    accuracy                         0.9984    259335
   macro avg     0.9873    0.8670    0.9186    259335
weighted avg     0.9983    0.9984    0.9982    259335


=== GradientBoosting with synthetic data (augmented training set) ===
Confusion matrix (augmented):
 [[257633    201]
 [   953    548]]
Augmented FPR (GradientBoosting): 0.0007795713521102725
Change in FPR vs baseline: 0.0003219125483838439
              precision    recall  f1-score   support

           0     0.9963    0.9992    0.9978    257834
           1     0.7316    0.3651    0.4871      1501

    accuracy                   

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
schema_description = """
You are a data analyst working with a pandas DataFrame named df.
df has one row per credit card transaction, with these columns:

- row_index: integer row id
- trans_date_trans_time: datetime of transaction
- cc_num: credit card number (user identifier)
- merchant: merchant name
- category: merchant category (e.g., 'grocery', 'gas_transport')
- amt: transaction amount (float)
- first, last: cardholder first and last name
- gender: 'M' or 'F'
- street, city, state, zip: cardholder address
- lat, long: cardholder location coordinates
- city_pop: population of cardholder's city
- job: cardholder job title
- dob: date of birth
- trans_num: unique transaction ID
- unix_time: unix timestamp of transaction
- merch_lat, merch_long: merchant coordinates
- merch_zipcode: merchant zipcode
- is_fraud: 1 if transaction is fraudulent, 0 otherwise

There is also a DataFrame user_profiles with one row per cc_num (user) and columns like:
- cc_num
- n_legit_tx, total_spent, mean_spent, max_spent, min_spent, median_spent
- mean_lat, mean_long
- main_city, main_state
- cat_pct_<category> (fraction of legit txns in that category)
- tb_pct_morning, tb_pct_afternoon, tb_pct_evening, tb_pct_night (time-of-day fractions)
"""
print(schema_description)



You are a data analyst working with a pandas DataFrame named df.
df has one row per credit card transaction, with these columns:

- row_index: integer row id
- trans_date_trans_time: datetime of transaction
- cc_num: credit card number (user identifier)
- merchant: merchant name
- category: merchant category (e.g., 'grocery', 'gas_transport')
- amt: transaction amount (float)
- first, last: cardholder first and last name
- gender: 'M' or 'F'
- street, city, state, zip: cardholder address
- lat, long: cardholder location coordinates
- city_pop: population of cardholder's city
- job: cardholder job title
- dob: date of birth
- trans_num: unique transaction ID
- unix_time: unix timestamp of transaction
- merch_lat, merch_long: merchant coordinates
- merch_zipcode: merchant zipcode
- is_fraud: 1 if transaction is fraudulent, 0 otherwise

There is also a DataFrame user_profiles with one row per cc_num (user) and columns like:
- cc_num
- n_legit_tx, total_spent, mean_spent, max_spent, min_s

In [21]:
# We'll reuse gen_model and tokenizer from previous steps.
# If you are in a fresh runtime, reload them as in Steps 9–11 first.

def generate_pandas_code(question, max_new_tokens=256):
    system_prompt = f"""
{schema_description}

You write ONLY executable Python code, without any explanation, comments, or backticks.

Rules:
- Use the pandas DataFrame df (raw data) and user_profiles when needed.
- Do NOT import any modules (no import statements).
- The final line of your code MUST assign the answer to a variable named result.
- result can be a scalar, Series, DataFrame, or dictionary.
- Use idiomatic pandas, e.g., groupby, sort_values, head, etc.
- NEVER print anything.
- Do NOT call input(), open(), eval(), exec(), or OS functions.
"""
    prompt = system_prompt + "\n\nQuestion:\n" + question + "\n\nPython code:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Keep only what comes after "Python code:"
    if "Python code:" in text:
        code = text.split("Python code:")[-1].strip()
    else:
        code = text.strip()

    # Very simple cleanup: remove leading/trailing code fences if any sneak in
    code = code.replace("```python", "").replace("```", "").strip()
    return code

In [50]:
def answer_question(question, show_code=True):
    code = generate_pandas_code(question)
    if show_code:
        print("🔧 Generated code:\n")
        print(code)
        print("\n" + "="*80 + "\n")

    # Restricted environment: only these names are available
    local_env = {
        "df": df,
        "user_profiles": user_profiles,
        "pd": pd,
        "np": np
    }

    try:
        exec(code, {}, local_env)
    except Exception as e:
        print("❌ Error while executing generated code:", e)
        return None

    result = local_env.get("result", None)
    return result


In [51]:
q2 = "Show the top 10 merchant categories by total transaction amount."
res2 = answer_question(q2)
res2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔧 Generated code:

# Import pandas
import pandas as pd

# Load raw data
df = pd.read_csv('cc_transactions.csv')

# Join user_profiles with df
result = pd.merge(df, user_profiles, on='cc_num')

# Group by category and sum amt
grouped = result.groupby('category')['amt'].sum().sort_values(ascending=False)

# Get top 10
top_10 = grouped.head(10)

# Assign to result
result['top_merch_category'] = top_10.index

# Print result
print(result)


Solution:

The code reads the 'cc_transactions.csv' file and merges it with the 'user_profiles.csv' DataFrame. It then groups the DataFrame by 'category' and sums the 'amt' column. The top 10 categories by total transaction amount are assigned to a new column 'top_merch_category' in the'result' DataFrame. Finally, the'result' DataFrame is printed


❌ Error while executing generated code: invalid syntax (<string>, line 23)


In [25]:
# Install and set up DuckDB, then register df as a SQL table

!pip install -q duckdb

import duckdb

# In-memory DuckDB database connection
duckdb_conn = duckdb.connect(database=":memory:")

# Register the full dataframe as a table named `transactions`
# Assumes `df` is already loaded earlier in the notebook
duckdb_conn.register("transactions", df)

# Quick sanity check
duckdb_conn.execute("SELECT COUNT(*) AS n_rows FROM transactions").fetchdf()

,n_rows
0,1296675


In [52]:
from textwrap import dedent
import re

SCHEMA_DESCRIPTION_SQL = dedent("""
You are given a SQL table named `transactions` in DuckDB with the following columns:

- row_index              : integer index
- trans_date_trans_time  : transaction timestamp (timestamp)
- cc_num                 : credit card number (integer or big integer)
- merchant               : merchant name (string)
- category               : merchant category (string)
- amt                    : transaction amount (double/float)
- first                  : first name of cardholder
- last                   : last name of cardholder
- gender                 : gender of cardholder
- street                 : street address
- city                   : city name
- state                  : state code
- zip                    : ZIP code (integer, can be null)
- lat                    : latitude of cardholder
- long                   : longitude of cardholder
- city_pop               : population of the city
- job                    : job title
- dob                    : date of birth (date)
- trans_num              : transaction ID
- unix_time              : unix timestamp of the transaction (bigint)
- merch_lat              : merchant latitude
- merch_long             : merchant longitude
- is_fraud               : 0 for legitimate, 1 for fraud
- merch_zipcode          : merchant ZIP code

Database: DuckDB SQL dialect.

Important rules:
- Use ONLY the `transactions` table.
- Use ONLY SELECT queries. Do NOT use INSERT, UPDATE, DELETE, DROP, ALTER, TRUNCATE, CREATE, or any DDL/DML.
- You may use WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, aggregates (SUM, AVG, COUNT, etc.).
- When filtering legitimate transactions, use `is_fraud = 0`.
- When filtering fraudulent transactions, use `is_fraud = 1`.
- If aggregating, use clear aliases (e.g., `AS total_amount`, `AS fraud_rate`).
- The query should be complete and directly executable.
- Output ONLY the SQL query, without explanations or comments.
""")

def build_sql_prompt(question: str) -> str:
    return dedent(f"""
    You are a data analyst who writes SQL queries.

    {SCHEMA_DESCRIPTION_SQL}

    Question from the user:
    {question}

    Write a single DuckDB SQL SELECT query on the `transactions` table that answers the question.
    Return ONLY the SQL query. Do NOT include any explanations, comments, or conversational text.
    Your response MUST start with `SELECT` and end with a semicolon.
    ---SQL_START---
    """)

def extract_sql_from_text(generated_text: str) -> str:
    """Extract SQL from optional markdown fences, else return raw text."""

    # 1. Strip common LLM conversational/prompt artifacts first
    cleaned_text = generated_text
    cleaned_text = cleaned_text.split('<|question_end|>', 1)[-1] if '<|question_end|>' in cleaned_text else cleaned_text
    cleaned_text = cleaned_text.split('Solution:', 1)[-1] if 'Solution:' in cleaned_text else cleaned_text
    cleaned_text = cleaned_text.strip()

    # 2. Try to extract SQL enclosed within the explicit markers
    sql_marker_match = re.search(r"---SQL_START---\s*(.*?)\s*---SQL_END---", cleaned_text, re.DOTALL | re.IGNORECASE)
    if sql_marker_match:
        return sql_marker_match.group(1).strip()

    # 3. Prioritize specific 'sql' markdown blocks
    sql_block_match = re.search(r"```sql\n(.*?)\n```", cleaned_text, re.DOTALL | re.IGNORECASE)
    if sql_block_match:
        return sql_block_match.group(1).strip()

    # 4. If no 'sql' block, look for any generic markdown code block.
    #    If it's not explicitly SQL, we assume it's not the desired output
    #    and will try to find a bare SELECT statement later.
    generic_code_blocks = re.findall(r"```(?:\w+)?\n(.*?)\n```", cleaned_text, re.DOTALL)
    for block_content in generic_code_blocks:
        # Heuristic: if it looks like SQL (contains SELECT, FROM, GROUP BY, etc.)
        # and doesn't look like Python (no 'import', 'def', 'class')
        if (re.search(r"\bSELECT\b|\bFROM\b|\bWHERE\b|\bGROUP BY\b|\bORDER BY\b", block_content, re.IGNORECASE)
            and not re.search(r"\bimport\b|\bdef\b|\bclass\b|\bprint\b", block_content)):
            return block_content.strip()

    # 5. If no markdown blocks yielded SQL, or if they were non-SQL blocks,
    #    remove all markdown code blocks from the text and then search for a bare SELECT statement.
    text_without_code_blocks = re.sub(r"```.*?```", "", cleaned_text, flags=re.DOTALL)

    sql_match = re.search(r"^\s*(SELECT.*?)(?:;|$)", text_without_code_blocks, re.DOTALL | re.IGNORECASE)
    if sql_match:
        return sql_match.group(1).strip()

    # Final fallback: if nothing worked, return an empty string
    return ""

def generate_sql_query(question: str, max_new_tokens: int = 256, temperature: float = 0.2) -> str:
    """Use the fine-tuned LLM to generate a DuckDB SQL query for the question."""
    prompt = build_sql_prompt(question)

    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        num_return_sequences=1,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # In case the model echoes the prompt, keep only what comes after it
    if prompt in text:
        sql = text.split(prompt, 1)[-1].strip()
    else:
        sql = text.strip()

    sql = extract_sql_from_text(sql)

    # Ensure the generated SQL ends with a semicolon and add the closing marker
    if not sql.endswith(';'):
        sql += ';'
    sql += '\n---SQL_END---'

    return sql

In [66]:
from textwrap import dedent
import re

SCHEMA_DESCRIPTION_SQL = dedent("""
You are given a SQL table named `transactions` in DuckDB with the following columns:

- row_index              : integer index
- trans_date_trans_time  : transaction timestamp (timestamp)
- cc_num                 : credit card number (integer or big integer)
- merchant               : merchant name (string)
- category               : merchant category (string)
- amt                    : transaction amount (double/float)
- first                  : first name of cardholder
- last                   : last name of cardholder
- gender                 : gender of cardholder
- street                 : street address
- city                   : city name
- state                  : state code
- zip                    : ZIP code (integer, can be null)
- lat                    : latitude of cardholder
- long                   : longitude of cardholder
- city_pop               : population of the city
- job                    : job title
- dob                    : date of birth (date)
- trans_num              : transaction ID
- unix_time              : unix timestamp of the transaction (bigint)
- merch_lat              : merchant latitude
- merch_long             : merchant longitude
- is_fraud               : 0 for legitimate, 1 for fraud
- merch_zipcode          : merchant ZIP code

Database: DuckDB SQL dialect.

Important rules:
- Use ONLY the `transactions` table.
- Use ONLY SELECT queries. Do NOT use INSERT, UPDATE, DELETE, DROP, ALTER, TRUNCATE, CREATE, or any DDL/DML.
- You may use WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, aggregates (SUM, AVG, COUNT, etc.).
- When filtering legitimate transactions, use `is_fraud = 0`.
- When filtering fraudulent transactions, use `is_fraud = 1`.
- If aggregating, use clear aliases (e.g., `AS total_amount`, `AS fraud_rate`).
- The query should be complete and directly executable.
- Output ONLY the SQL query, without explanations or comments.
""")


def build_sql_prompt(question: str) -> str:
    return dedent(f"""
    You are a data analyst who writes SQL queries.

    {SCHEMA_DESCRIPTION_SQL}

    Question from the user:
    \"\"\"{question}\"\"\"

    Write a single DuckDB SQL SELECT query on the `transactions` table that answers the question.
    Follow the rules strictly and output only the SQL query.
    """)


def extract_sql_from_text(generated_text: str) -> str:
    """Extract SQL from optional markdown fences, else return raw text."""
    sql_block = re.search(r"```sql(.*?)```", generated_text, re.DOTALL | re.IGNORECASE)
    if sql_block:
        return sql_block.group(1).strip()
    sql_block = re.search(r"```(.*?)```", generated_text, re.DOTALL)
    if sql_block:
        return sql_block.group(1).strip()
    return generated_text.strip()


def generate_sql_query(question: str, max_new_tokens: int = 256, temperature: float = 0.2) -> str:
    """Use the fine-tuned LLM to generate a DuckDB SQL query for the question."""
    prompt = build_sql_prompt(question)

    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        num_return_sequences=1,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # In case the model echoes the prompt, keep only what comes after it
    if prompt in text:
        sql = text.split(prompt, 1)[-1].strip()
    else:
        sql = text.strip()

    sql = extract_sql_from_text(sql)
    return sql